In [ ]:
# LIB INSTALLATION

!pip install yfinance==0.2.41
!pip install crewai==0.28.8
!pip install 'crewai[tools]'
!pip install langchain==0.1.20
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.0.38
!pip install duckduckgo-search==5.3.0

In [71]:
# LIB IMPORTS

import json
import os
from datetime import datetime, date

import yfinance as yf

import numpy as np
np.float_ = np.float64

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [58]:
# CREATE YAHOO FINANCE TOOL

today = date.today()
last_year = str(datetime.now().year - 1) + "-01-01"

def fetch_stock_price(ticket):
  stock = yf.download(ticket, start=last_year, end=today)

  return stock

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [59]:
# OPENAI LLM - GPT IMPORT

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [60]:
stockPriceAnalyst = Agent(
  role = "Senior stock price Analyst",
  goal = "Find the {ticket} stock price and analyses trends",
  backstory = """You're highly experienced in analyzing the price of a 
  specific stock and make predictions about its future price.""",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = False,
  tools = [yahoo_finance_tool],
)

In [61]:
getStockPrice = Task(
  description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
  expected_output = """Specify the current trend stock price - up, down or sideways.
  eg. stock = 'APPL, price UP' 
  """,
  agent = stockPriceAnalyst
)

In [62]:
# SEARCH TOOL IMPORT

search_tool = DuckDuckGoSearchResults(backend = 'news' , num_results = 10)

In [63]:
newsAnalyst = Agent(
  role = "Stock News Analyst",
  goal = """Create a short summary of the market news related to the stock 
  {ticket} company. Specify the current trend - up, down or sideways with the news
  context. For each request stock asset, specify a number between 0 and 100, where 0 is
  extreme fear and 100 is extreme greed.
  """,
  backstory = """You're highly experienced in analyzing the market trends and news and have
  tracked assert for more then 10 years.

  You're also master level analysts in the tradicional markets and have deep understanding
  of human psychology.

  You understand news, theirs titles and information, but you look at those with
  a health dose of skepticism. 
  You consider also the source of the news articles.
  """,
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  allow_delegation = False,
  tools = [search_tool],
)

In [64]:
get_news = Task(
  description = """Take the stock and always include BTC to it (if not request).
  Use the search tool to search each one individually.

  The current date is {datetime}.

  Compose the results into a helpful report""",
  expected_output = """A summary of the overall market and one sentence summary 
  for each request asset.
  Include a fear/greed score for each asset based on the news. Use format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent = newsAnalyst
)

In [65]:
stockAnalystWrite = Agent(
  role = "Senior Stock Analyst Writer",
  goal = """Analyze the trends price and news and write an insightful compelling and informative 3 paragraph long newsletter
  based on the stock report and price trend.
  """,
  backstory = """You're widely accepted as the best stock analyst in the market. 
  You understand complex concepts and create compelling stories and narratives
  that resonate with wider audiences.

  You understand macro factors and comine multiple theories - eg. cycle theory and 
  fundamental analyses. You're able to hold multiple opinions when analyzing anything.
  """,
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = True,
)

In [66]:
writeAnalyses = Task(
  description = """Use the stock price trend and the news stock news report to
  create ab analyses and write the newsletter about the {ticket} company that is
  brief and highlights the most important points.
  Focus on the stock price trend, news and fear/greed score. What are the near 
  future considerations?
  Include the previous analyses of stock trend and news summary.
  """,
  expected_output = """An eloquent 3 paragraphs newsletter formatted as markdown
  in an easy readable manner. It should contain:

  - 3 bullets executive summary
  - Introduction - set the overall picture and spike up the interest
  - main part provides the meat of the analysis including the news summary and 
  fear/greed scores.
  - summary - key facts and concrete future trend prediction - up, down or sideways.
  """,
  agent = stockAnalystWrite,
  context = [getStockPrice, get_news]
)

In [67]:
crew = Crew(
  agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
  tasks = [getStockPrice, get_news, writeAnalyses],
  # verbose = 2,
  verbose = True,
  process = Process.hierarchical,
  full_output = True,
  share_crew = False,
  manager_llm = llm,
  max_iter = 15
)

2024-08-21 10:04:49,896 - 137505780768768 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
results = crew.kickoff(inputs={'ticket': 'AAPL', 'datetime': '2024-08-21'})

In [ ]:
list(results.keys())

In [ ]:
results['final_output']

In [ ]:
len(results['tasks_outputs'])

In [ ]:
Markdown(results['final_output'])